In [ ]:
import numpy as np
from netCDF4 import Dataset
from wrf import vinterp, getvar
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from bwave_ideal_wrf import qgpv_pert

### Define constants
nx = 2000
ny = 1800
nz = 90
ti = 4
hres = 4
G = 9.81
F0 = 1.0e-4
zbot = 0.2
ztop = 18
z = np.linspace(zbot,ztop,nz)

In [ ]:
### Read in WRF data and interpolate
nc1 = Dataset('/p/work1/lloveras/adj_4km/long_runs/ctl_48h/wrfout_d01_2021-01-05_00_00_00')
nc2 = Dataset('/p/work1/lloveras/adj_4km/long_runs/wave_48h_100/wrfout_d01_2021-01-05_00_00_00')

pv1 = np.asarray(getvar(nc1,'pvo',timeidx=ti))*1e-6
pv2 = np.asarray(getvar(nc2,'pvo',timeidx=ti))*1e-6

p1 = np.asarray(getvar(nc1,'p',units='Pa',timeidx=ti))
p2 = np.asarray(getvar(nc2,'p',units='Pa',timeidx=ti))

th1 = np.asarray(getvar(nc1,'th',timeidx=ti))
th2 = np.asarray(getvar(nc2,'th',timeidx=ti))

pv1 = vinterp(nc1,pv1,'ght_msl',z,timeidx=ti)
pv2 = vinterp(nc2,pv2,'ght_msl',z,timeidx=ti)

p1 = vinterp(nc1,p1,'ght_msl',z,timeidx=ti)
p2 = vinterp(nc2,p2,'ght_msl',z,timeidx=ti)

th1 = vinterp(nc1,th1,'ght_msl',z,timeidx=ti)
th2 = vinterp(nc2,th2,'ght_msl',z,timeidx=ti)


In [ ]:
### Compute background fields
dp_dt1 = np.zeros((nz,ny,nx))
dp_dt2 = np.zeros((nz,ny,nx))
dt_dz = np.zeros((nz-1,ny,nx))
n = np.zeros((nz-1,ny,nx))
for k in range(1,nz-1):
    dp_dt1[k,:,:] = (p1[k+1,:,:] - p1[k-1,:,:])/(th1[k+1,:,:] - th1[k-1,:,:])
    dp_dt2[k,:,:] = (p2[k+1,:,:] - p2[k-1,:,:])/(th2[k+1,:,:] - th2[k-1,:,:])
    dt_dz[k,:,:] = (th1[k+1,:,:] - th1[k-1,:,:])/((z[k+1] - z[k-1])*1000.)
    n[k,:,:] = np.sqrt(G/th1[k,:,:]*dt_dz[k,:,:])
    
dp_dt1[0,:,:] = dp_dt1[1,:,:]
dp_dt1[-1,:,:] = (p1[-1,:,:] - p1[-2,:,:])/(th1[-1,:,:] - th1[-2,:,:])
dp_dt2[0,:,:] = dp_dt2[1,:,:]
dp_dt2[-1,:,:] = (p2[-1,:,:] - p2[-2,:,:])/(th2[-1,:,:] - th2[-2,:,:])
dt_dz[0,:,:] = dt_dz[1,:,:]
n[0,:,:] = np.sqrt(G/th1[0,:,:]*dt_dz[0,:,:])

sig1 = (-1/G)*dp_dt1
sig2 = (-1/G)*dp_dt2
dtdz_avg = np.nanmean(dt_dz,axis=(1,2))
n_avg = np.nanmean(n,axis=(1,2))
bu = n_avg/F0
bu_fac = bu**2


In [ ]:
### Compute difference fields
dqgpv = sig2*pv2 - sig1*pv1

### Transform into spectral space
pvsp = np.zeros((nz,ny,nx),dtype=np.dtype(np.complex64))
for k in range(nz):
    pvsp[k,:,:] = np.fft.fft2(np.squeeze(dqgpv[k,:,:]))

### Invert
lbcxy = np.zeros((ny,nx))
ubcxy = np.zeros((ny,nx))
lbcsp = np.fft.fft2(lbcxy)
ubcsp = np.fft.fft2(ubcxy)
xl, yl, x, y, xg, yg, dz, zp, facz = qgpv_pert.cartesian_mesh(nx, ny, nz, hres, ztop)
kmax, lmax, facx, facy, dxg, dyg = qgpv_pert.spectral_mesh(nx, ny, xl, yl)


In [ ]:
fbsp, ftsp, fzbsp, fztsp, fsp = qgpv_pert.qgpv_inversion(nx, ny, nz, bu_fac, \
        facx, facy, facz, kmax, lmax, pvsp, ubcsp, lbcsp, dz)

du_qg, dv_qg, dth_qg, dr_qg, dphi_qg = qgpv_pert.qgpv_solver(fsp, nx, ny, nz, \
        bu_fac, dxg, dyg, dz, lbcxy, ubcxy, dtdz_avg)


In [ ]:
np.amax(dv_qg)